### SASA trajectory analysis.
Works well for protein and ligand residues of up to 10,000 atoms. You will need a pdb file and a trajectory file. Protein and ligand must have DIFFERENT CHAIN IDs. You also need to have naccess and MDAnalysis.
Written by Peter Obi 11/17/2022

In [13]:
import MDAnalysis

In [6]:
topology = 'step5_input.pdb'
trajectory = '5.trr' 
u = MDAnalysis.Universe(topology,trajectory)
protein_ligand = u.select_atoms("protein or (resname P1N)") #change P1N to ligand name
ligand = u.select_atoms("resname P1N")
len(u.trajectory)

6

In [7]:
for ts in u.trajectory:
    with MDAnalysis.Writer("{}_complex.pdb".format(ts.frame), protein_ligand.n_atoms) as W:
        W.write(protein_ligand)
    with MDAnalysis.Writer("{}_ligand.pdb".format(ts.frame), ligand.n_atoms) as L:
        L.write(ligand)     

In [ ]:
cmd = '''
cd /home/projects/petero/binding-pocket-project/naccess/test
for filename in /home/projects/petero/binding-pocket-project/naccess/test/*.pdb
do
  /home/projects/petero/software/naccess/naccess "$filename";
done
touch SASA_ligand.txt
touch SASA_complex.txt
for filename in test/*
do
  prefix="${filename##*/}"
  base="${prefix%%_*}"
  echo "$base"_ligand_sasa>>SASA_ligand.txt
  grep "CHAIN  1 H" ${base}_ligand.rsa >>SASA_ligand.txt
  echo "$base"_complex_sasa >>SASA_complex.txt
  grep "CHAIN  2 H" ${base}_complex.rsa >>SASA_complex.txt
done 
'''

In [ ]:
subprocess.run(cmd, shell=True, check=True, executable='/usr/bin/bash')

In [ ]:
df = pd.read_fwf('/home/projects/petero/binding-pocket-project/naccess/test/SASA_ligand.txt', skiprows=1, header = None)
df1 = pd.read_fwf('/home/projects/petero/binding-pocket-project/naccess/test/SASA_complex.txt', skiprows=1, header = None

In [ ]:
df

In [ ]:
df1

In [ ]:
df.columns= ['PDB ID', 'All-atoms', 'Total-Side', 'Main-Chain', 'Non-polar', 'All polar']
df1.columns= ['PDB ID','All-atoms', 'Total-Side', 'Main-Chain', 'Non-polar', 'All polar']

In [ ]:
df

In [ ]:
df1

In [ ]:
df2 = (pd.merge(df, df1, on='PDB ID'))

In [ ]:
df2

In [ ]:
df2['BSA_total'] = df2['All-atoms_x']- df2['All-atoms_y']
df2['MESA_total'] = df2['All-atoms_y']

In [ ]:
df2['BSA_nonpolar'] = df2['Non-polar_x']- df2['Non-polar_y']
df2['MESA_nonpolar'] = df2['Non-polar_y']

In [ ]:
df2['BSA_polar'] = df2['All polar_x']- df2['All polar_y']
df2['MESA_polar'] = df2['All polar_y']

In [ ]:
df2

In [ ]:
df2.to_csv('/home/projects/petero/binding-pocket-project/naccess/test/final_sasa.csv', index=False)